### Seq2Seq model for English to Chinese translation (baseline model)

In [17]:
import json
import numpy as np
import pickle
from collections import Counter

### Choosing, loading, and cleaning dataset

- going to use datasets from: https://www.kaggle.com/datasets/qianhuan/translation?resource=download

In [ ]:
train_set_path = "dataset/translation2019zh_train.json"

train_set = []
with open(train_set_path) as f:
    for line in f:
        train_set.append(json.loads(line))

print(len(train_set))
print(train_set[0])

5161434
{'english': 'For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.', 'chinese': '为了更好的锐度，但是附带的会多一些颗粒度，可以使用这个显影剂的1：1稀释液。'}


We want to lower the size of this dataset, for testing purposes.
- right now 5.1M sentences
- let's make it 10,000 sentences...

In [12]:
# get 10,000 random indices 
sampled_indices = np.random.choice(len(train_set), 10000)

train_subset = [train_set[i] for i in sampled_indices]
print(train_subset[0])
with open('dataset/train_set_mini.pkl', 'wb') as f:
    pickle.dump(train_subset, f)

{'english': 'The amphibian & reptile hall, the marine animal hall, the tropic monkey hall, the bird garden are to be built on the flat ground, and the Garden will welcome Chinese and foreign guests with new looks.', 'chinese': '两栖——爬行动物馆、海兽馆、热带猴馆、鸟园等新型建筑将平地而起，以新的容貌迎接中外宾客。'}


In [13]:
with open('dataset/train_set_mini.pkl', 'rb') as f:
    train_set_mini = pickle.load(f)

print(train_set_mini[0])

{'english': 'The amphibian & reptile hall, the marine animal hall, the tropic monkey hall, the bird garden are to be built on the flat ground, and the Garden will welcome Chinese and foreign guests with new looks.', 'chinese': '两栖——爬行动物馆、海兽馆、热带猴馆、鸟园等新型建筑将平地而起，以新的容貌迎接中外宾客。'}


Train set made. 
Now to work on the actual architecture

## Preprocessing steps:
- we want to maintain vocabulary for english and chinese. 
    - this is simple, just use a counter() and then limit it to if it appears within 5

- In terms of encoding, we want to use sequence input, so a sentence becomes a list [3, 100, 8, 9], where each number corresponds to the index of the word in the dictionary
    - we want to do this because LSTMs keep track of the word relationships at the sentence level
    - then use nn.Embedding?
        - nn.Embedding allows us to create a matrix representing the vocabulary. It allows us to train some nuance into the words, where instead of typical BoW where each word is just an index, each word is now a vector, which allows us to train some meaning into the word
        - the embedding is a matrix size (vocab length, dim). Length vocab length because each row in the matrix corresponds to a word in the vocab, ( row of index = index of word in vocab :) )
        

# now to work on the vocabulary

In [19]:
def get_en_dict(train_set):
    '''
    get_en_dict:
        Gets an english dict from train_set 
    '''
    new_l = [train_set[0], train_set[1]]
    c = Counter(new_l)
    print(c)
    return new_l
    

train_set_mini[0]['english']
print(get_en_dict(train_set_mini))

TypeError: unhashable type: 'dict'